Imports

In [ ]:
!pip install -Uqq fastai torch torchvision

import pandas as pd
import numpy as np
import torch
from fastai.tabular.all import (
    TabularPandas, FillMissing, Normalize, RandomSplitter,
    tabular_learner, mae, MSELossFlat
)
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.3/235.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156

DATA PREPARATION

In [ ]:
# Paths
base_dir = '/content/drive/MyDrive/rul_project'
train_file = f'{base_dir}/P_Train2.csv'
test_file  = f'{base_dir}/P_Test2.csv'

# Load and clean
train_df = pd.read_csv(train_file)
train_df.columns = train_df.columns.str.strip().str.lower()
print("Columns in train_df:", train_df.columns)  # DEBUG

test_df = pd.read_csv(test_file)
test_df.columns = test_df.columns.str.strip().str.lower()

# Check that 'rul' exists!
assert 'rul' in train_df.columns, "Your CSV is missing the 'rul' column."

cont_names = [c for c in train_df.columns if c != 'rul']
cat_names  = []

procs  = [FillMissing, Normalize]
splits = RandomSplitter(valid_pct=0.2, seed=42)(train_df)

tab = TabularPandas(
    train_df, procs=procs,
    cat_names=cat_names, cont_names=cont_names,
    y_names='rul', splits=splits
)
dls = tab.dataloaders(bs=32)


Columns in train_df: Index(['id', 'cycle', 'sm2', 'sm3', 'sm4', 'sm7', 'sm8', 'sm11', 'sm12',
       'sm13', 'sm15', 'sm17', 'sm20', 'sm21', 'rul'],
      dtype='object')


RECREATE MODEL AND LOAD BEST WEIGHTS

In [ ]:
layers = [400, 200, 100]
learn = tabular_learner(
    dls,
    layers=layers,
    loss_func=MSELossFlat(),
    metrics=mae
)

model_save_path = f'{base_dir}/best_fd002'
learn.save(model_save_path)
learn.load(model_save_path)


/usr/local/lib/python3.11/dist-packages/fastai/learner.py:67: UserWarning: Saved file doesn't contain an optimizer state.
  elif with_opt: warn("Saved file doesn't contain an optimizer state.")


EXPORT THE MODEL FOR FRONTEND DEVELOPERS

In [ ]:
# 1. Install packages (Colab cell)
!pip install -Uqq fastai torch torchvision

# 2. Imports
import pandas as pd
import numpy as np
import torch
from fastai.tabular.all import (
    TabularPandas, FillMissing, Normalize, RandomSplitter,
    tabular_learner, mae, MSELossFlat
)
from google.colab import drive
drive.mount('/content/drive')

# 3. Paths
base_dir = '/content/drive/MyDrive/rul_project'
train_file = f'{base_dir}/P_Train2.csv'
test_file  = f'{base_dir}/P_Test2.csv'
model_save_path = f'{base_dir}/best_fd002'  # DO NOT add .pth extension!

# 4. Load and clean data
train_df = pd.read_csv(train_file)
train_df.columns = train_df.columns.str.strip().str.lower()
test_df = pd.read_csv(test_file)
test_df.columns = test_df.columns.str.strip().str.lower()

assert 'rul' in train_df.columns, "Your CSV is missing the 'rul' column."
cont_names = [c for c in train_df.columns if c != 'rul']
cat_names  = []

# 5. Setup fastai TabularPandas and DataLoaders
procs  = [FillMissing, Normalize]
splits = RandomSplitter(valid_pct=0.2, seed=42)(train_df)
tab = TabularPandas(
    train_df, procs=procs,
    cat_names=cat_names, cont_names=cont_names,
    y_names='rul', splits=splits
)
dls = tab.dataloaders(bs=32)

# 6. Create learner
layers = [400, 200, 100]
learn = tabular_learner(
    dls,
    layers=layers,
    loss_func=MSELossFlat(),
    metrics=mae
)

# 7. Train model (uncomment and run for training, or skip if already trained)
lr = 0.015849
learn.fit_one_cycle(30, lr_max=lr)

# 8. Save model to Google Drive
learn.save(model_save_path)  # Saves as 'best_fd002.pth' in your Drive!

# 9. Load model from Google Drive (for future use, validation, or inference)
learn.load(model_save_path)  # Looks for 'best_fd002.pth' in your Drive

# 10. Confirm MAE on the validation set
val_mae = learn.validate()[1]
print(f"Validation MAE of loaded model: {val_mae:.4f}")

print('Model trained, saved, loaded, and validated successfully. Ready for inference or export!')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


epoch,train_loss,valid_loss,mae,time
0,1819.538330,1553.570312,29.463625,00:17
1,1694.686401,1635.078247,29.973770,00:17
2,1489.202271,1501.269775,28.728601,00:16
3,1675.884766,1717.552246,30.143536,00:16
4,1628.234985,1576.809448,28.708557,00:17
5,1615.343140,2152.038818,38.442047,00:18
6,1710.621582,1730.948608,32.585335,00:17
7,1755.802368,2436.211426,37.115005,00:17
8,1588.365479,2462.953125,31.031404,00:17
9,1601.012329,1579.274536,29.062504,00:20


Validation MAE of loaded model: 27.6256
Model trained, saved, loaded, and validated successfully. Ready for inference or export!


In [ ]:
# --- 11. Calculate Healthy Ranges for Each Sensor at Each RUL Bin ---

# Parameters
RUL_BIN_SIZE      = 10      # bin width in cycles (0–10,10–20,…)
PCTL_LOW, PCTL_HI = 10, 90  # percentile bounds for "healthy" range

# Copy and bin
train_health = train_df.copy()
train_health['rul_bin'] = (train_health['rul']//RUL_BIN_SIZE)*RUL_BIN_SIZE

# Identify only the sensor columns (drop metadata)
sensor_cols = [c for c in cont_names
               if c not in ('cycle','id','time') and not c.startswith('engine')]

# Compute per-bin low/high percentiles
agg_dict = {c: [lambda x: np.percentile(x,PCTL_LOW),
                lambda x: np.percentile(x,PCTL_HI)]
            for c in sensor_cols}
healthy_ranges = (
    train_health
    .groupby('rul_bin')
    .agg(agg_dict)
)
# Flatten column names
healthy_ranges.columns = [f"{sensor}_{suffix}"
                          for sensor,suffix in healthy_ranges.columns]
healthy_ranges = healthy_ranges.reset_index()
print("Healthy ranges sample:")
print(healthy_ranges.head())

# === ADD THIS LINE to save as CSV ===
healthy_ranges.to_csv(f"{base_dir}/healthy_ranges_fd002.csv", index=False)
print(f"Saved healthy ranges to {base_dir}/healthy_ranges_fd002.csv")



# --- 12. Helper: fetch the low/high band for a predicted RUL ---
def get_healthy_band(pred_rul):
    bin_val = int((pred_rul//RUL_BIN_SIZE)*RUL_BIN_SIZE)
    sub = healthy_ranges[healthy_ranges.rul_bin==bin_val]
    if sub.empty:
        # fallback to nearest
        i = (healthy_ranges.rul_bin - bin_val).abs().argmin()
        sub = healthy_ranges.iloc[[i]]
    out = {}
    for s in sensor_cols:
        low  = sub[f"{s}_<lambda_0>"].values[0]
        high = sub[f"{s}_<lambda_1>"].values[0]
        out[s] = (low,high)
    return out


# --- 13. Inference + comparison function using test_dl() ---
def predict_and_compare(row):
    """
    row: a pd.Series or dict containing at least all `cont_names`
    """
    # 1. Build single-row DataFrame with only our cont_names
    df = pd.DataFrame([row])[cont_names]
    # 2. Create a test dataloader and get preds
    dl = learn.dls.test_dl(df)
    preds,_ = learn.get_preds(dl=dl)
    pred_rul = preds[0].item()
    print(f"\nPredicted RUL: {pred_rul:.1f}")

    # 3. Get the healthy band for that RUL
    band = get_healthy_band(pred_rul)
    print("\nSensor\tValue\tLow\tHigh\tOK?")
    for s in sensor_cols:
        val = row[s]
        low,high = band[s]
        ok = low <= val <= high
        print(f"{s}\t{val:.2f}\t{low:.2f}\t{high:.2f}\t{'In range' if ok else 'Warning!'}")


# --- 14. Example on your test set ---
sample = test_df.iloc[0]  # pd.Series
predict_and_compare(sample)


Healthy ranges sample:
   rul_bin  sm2_<lambda_0>  sm2_<lambda_1>  sm3_<lambda_0>  sm3_<lambda_1>  \
0        0         537.510         643.460        1272.838        1599.512   
1       10         537.340         643.250        1270.875        1596.523   
2       20         537.239         643.030        1269.258        1594.216   
3       30         537.180         642.820        1267.607        1592.418   
4       40         537.150         642.801        1267.448        1591.360   

   sm4_<lambda_0>  sm4_<lambda_1>  sm7_<lambda_0>  sm7_<lambda_1>  \
0        1065.269        1425.991          138.12          551.41   
1        1062.193        1420.625          138.25          551.85   
2        1059.636        1416.751          138.33          552.20   
3        1057.198        1413.230          138.36          552.39   
4        1056.279        1411.509          138.38          552.66   

   sm8_<lambda_0>  ...  sm13_<lambda_0>  sm13_<lambda_1>  sm15_<lambda_0>  \
0        1916.01


Predicted RUL: 190.9

Sensor	Value	Low	High	OK?
sm2	605.03	536.81	642.15	In range
sm3	1497.17	1262.20	1584.33	In range
sm4	1304.99	1047.41	1399.63	In range
sm7	394.54	138.61	553.68	In range
sm8	2318.96	1915.39	2388.03	In range
sm11	45.61	36.67	47.25	In range
sm12	371.69	130.55	521.66	In range
sm13	2388.18	2028.31	2388.16	Warning!
sm15	8.65	8.43	10.87	In range
sm17	369.00	307.00	392.00	In range
sm20	28.42	10.59	38.89	In range
sm21	17.16	6.35	23.33	In range
